# Imports

In [12]:
import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns
import warnings

import joblib

import psycopg2 as ps

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

import os

In [13]:
PATH_IGNORE = 'ignore'
PATH_DATA = 'data'
FILE_DATA = os.path.join(PATH_DATA, 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx')
FILE_OLD_DATA = os.path.join(PATH_DATA, 'PEDE_PASSOS_DATASET_FIAP_ANTIGO.xlsx')

In [14]:
df = {}

df[2022] = pd.read_excel(FILE_DATA, sheet_name=0)
df[2023] = pd.read_excel(FILE_DATA, sheet_name=1)
df[2024] = pd.read_excel(FILE_DATA, sheet_name=2)

# Dicionário

**INDE** - Índice de desenvolvimento educacional
- Ponderação entre os indicadores abaixo

**IAN** - Indicador de adequação de nível
- Registros administrativos
- Registra a defasagem da aprendizagem do aluno por meio da comparação da Fase atual do aluno da Associação, com a equivalência das Fases de ensido na Associação e a divisão dos anos escolares do ensino formal (por idade)
- Utiliza a fase atual, e a fase ideal conforme a idade.
- 10 (em fase), 5 (moderada) ou 2,5 (severa).

**IDA** - Indicador de desempenho acadêmico
- Notas Provas PM e Média geral Universitária
- Registra a proficiência do aluno nos exames padronizados de avaliação interna das disciplinas oferecidas pela Associação (os alunos das Fases 0 a 7). No caso dos alunos da Fase 8, esse indicador registra a nota média obtida pelos alunos em todas as disciplinas curriculares cursadas, nas respectivas instituições de ensino superior conveniadas ao programa de bolsas de estudo da Associação.
- Utiliza as notas internas da associação. Matemática + Português + Inglês / 3
- 0 a 10

**IEG** - Indicador de engajamento
- Registros de entrega de lição de casa e de voluntariado
- Mede o engajamento do aluno nas tarefas curriculares requeridas em cada uma das disciplinas oferecidas na Associação, e/ou seu engajamento em ações de voluntariado desenvolvidas pela Associação.
- Soma das pontuações das tarefas realizadas e registradas / Número de tarefas

**IAA** - Indicador de Autoavaliação
- Questionário de Autoavaliação individual
- Registra por meio de um questionário padronizado e adaptado às distintas faixas etárias dos alunoa a Associação, uma autoavaliação do aluno sobre como se sente consigo mesmo, sobre os estudos, sobre sua família, amigos e comunidade, e e sobre como se sente a respeito da Associação Passos Mágicos.
- Soma das pontuações das respostas do estudante / Número total de perguntas

**IPS** - Indicador Psicossocial
- Questionário individual de avaliação das psicólogas
- Avaliação da equipe de psicólogas para caracterizar o desenvolvimento do aluno nas suas interações familiares, no seu desenvolvimento emocional, comportamental e da sua socialização na vida comunitária. Esse indicador também caracteriza o tipo de atendimento psicológico oferecido pela Associação ao aluno.
- Avaliações feitas por **psicólogos**.
- Soma das pontuações / Número de avaliadores

**IPP** - Indicador Psicopedagógico
- Questionário individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos para caracterizar o desenvolvimento cognitivo, emocional, comportamental e de socialização do aluno no seu processo de aprendizado dentro do Programa de Aceleração do Conhecimento, dos Programas Educacionais e das Atividades Culturais promovidas pela Associação.
- Soma das avaliações sobre aspectos **pedagógicos** / Número de avaliações

**IPV** - Indicador de Ponto de Virada
- Questionário Individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos a respeito do desenvolvimento do aluno das aptidões necessárias para iniciar a transformação da sua vida por meio da Educação, avaliando a integração do aluno à Associação, o seu desenvolvimento emocional, e o seu potencial acadêmico.
- Análises longitudinais de progresso acadêmico, engajamento e desenvolvimento emocional

**Pedras** - Baseadas no INDE
- Quartzo: 3.0 < INDE < 6.1
- Ágata: 6.1 < INDE < 7.2
- Ametista: 7.2 < INDE < 8.2
- Topázio: 8.2 < INDE < 9.4

# Padronizando colunas
Algumas colunas estão nomeadas diferentemente entre anos. Vamos arrumar isso antes de proceder.

In [15]:
for year in df:
    print(df[year].columns)

Index(['RA', 'Fase', 'Turma', 'Nome', 'Ano nasc', 'Idade 22', 'Gênero',
       'Ano ingresso', 'Instituição de ensino', 'Pedra 20', 'Pedra 21',
       'Pedra 22', 'INDE 22', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'Rec Psicologia', 'IDA',
       'Matem', 'Portug', 'Inglês', 'Indicado', 'Atingiu PV', 'IPV', 'IAN',
       'Fase ideal', 'Defas', 'Destaque IEG', 'Destaque IDA', 'Destaque IPV'],
      dtype='object')
Index(['RA', 'Fase', 'INDE 2023', 'Pedra 2023', 'Turma', 'Nome Anonimizado',
       'Data de Nasc', 'Idade', 'Gênero', 'Ano ingresso',
       'Instituição de ensino', 'Pedra 20', 'Pedra 21', 'Pedra 22', 'Pedra 23',
       'INDE 22', 'INDE 23', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1',
       'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3',
       'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'IPP', 'Rec Psicologia',
       'IDA', 'Mat', 'Por', 'In

In [16]:
def check_column(
        df : pd.DataFrame,
        column : str
):
    ''' Checks if a column exists in a DataFrame '''
    if column in df.columns:
        return True
    else: 
        return False

In [17]:
def check_column_in_many(
        df_list : list,
        column : str,
        verbose : bool = False
):
    '''Checks if a column exists in all given DataFrames'''
    exists = True
    count = 0
    for df in df_list:
        column_exists = check_column(df, column)
        if column_exists != True:
            exists = False
            if verbose:
                print(f'Column does not exists in dataframe {count}')
        count += 1
    return exists

In [18]:
# Colunas que queremos no nossos dataframes padronizados
universal_columns = [
    'Nome Anonimizado',
    'Fase',
    'Idade',
    'Gênero',
    'Ano ingresso',
    'IAN',
    'IDA',
    'IEG',
    'IAA',
    'IPS',
    'IPP',
    'IPV',
    # 'Atingiu PV',
    'Mat',
    'Por',
    'Ing',
    'Instituição de ensino'
]

In [19]:
uneven_columns = []

for column in universal_columns:
    exists_in_all = check_column_in_many(
        df_list = list(df.values()),
        column = column
    )

    if exists_in_all == False:
        uneven_columns.append(column)

uneven_columns

['Nome Anonimizado', 'Idade', 'IPP', 'Mat', 'Por', 'Ing']

In [20]:
df[2022] = df[2022].rename(
    {'Nome' : 'Nome Anonimizado',
     'Idade 22' : 'Idade',
     'Matem' : 'Mat',
     'Portug': 'Por',
     'Inglês' : 'Ing'},
     axis = 'columns',
     errors = 'raise'
)

In [21]:
uneven_columns = []

for column in universal_columns:
    # print(f'Checking {column}')
    exists_in_all = check_column_in_many(
        df_list = list(df.values()),
        column = column
    )

    if exists_in_all == False:
        uneven_columns.append(column)
    # print(exists_in_all)
    # print()
    
uneven_columns

['IPP']

In [22]:
df[2023][universal_columns]

,Nome Anonimizado,Fase,Idade,Gênero,Ano ingresso,IAN,IDA,IEG,IAA,IPS,IPP,IPV,Mat,Por,Ing,Instituição de ensino
0,Aluno-861,ALFA,8,Feminino,2023,10.0,9.6,10.0,9.5,8.13,8.4375,8.920,9.8,9.4,NaN,Pública
1,Aluno-862,ALFA,9,Masculino,2023,5.0,8.9,9.1,8.5,8.14,7.5000,8.585,8.5,9.2,NaN,Pública
2,Aluno-863,ALFA,7,Masculino,2023,10.0,6.3,7.6,0.0,3.14,5.9375,6.260,7.0,5.5,NaN,Pública
3,Aluno-864,ALFA,1900-01-08 00:00:00,Feminino,2023,10.0,6.3,7.6,0.0,8.14,7.5000,8.500,7.0,5.5,NaN,Pública
4,Aluno-865,ALFA,8,Masculino,2023,10.0,7.4,8.7,8.5,7.52,7.5000,7.915,7.3,7.5,NaN,Pública
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,Aluno-2,FASE 8,18,Feminino,2017,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privada *Parcerias com Bolsa 100%
1010,Aluno-1271,FASE 8,18,Feminino,2021,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privada *Parcerias com Bolsa 100%
1011,Aluno-1272,FASE 8,1900-01-18 00:00:00,Feminino,2021,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privada *Parcerias com Bolsa 100%
1012,Aluno-1273,FASE 8,17,Feminino,2023,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privada - Pagamento por *Empresa Parceira


# 1. Adequação do nível (IAN): Qual é o perfil geral de defasagem dos alunos (IAN) e como ele evolui ao longo do ano?
(Ex.: quantos alunos estão moderadamente ou severamente defasados?)

# 2. Desempenho acadêmico (IDA): O desempenho acadêmico médio (IDA) está melhorando, estagnado ou caindo ao longo das fases e anos?

# 3. Engajamento nas atividades (IEG): O grau de engajamento dos alunos (IEG) tem relação direta com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)?

# 4. Autoavaliação (IAA): As percepções dos alunos sobre si mesmos (IAA) são coerentes com seu desempenho real (IDA) e engajamento (IEG)?

# 5. Aspectos psicossociais (IPS): Há padrões psicossociais (IPS) que antecedem quedas de desempenho acadêmico ou de engajamento?

# 6. Aspectos psicopedagógicos (IPP): As avaliações psicopedagógicas (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?

# 7. Ponto de virada (IPV): Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?

# 8. Multidimensionalidade dos indicadores: Quais combinações de indicadores (IDA + IEG + IPS + IPP) melhor explicam o desempenho global do aluno (INDE)?

# 9. Previsão de risco com Machine Learning: Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem? Construa um modelo preditivo que mostre uma probabilidade do aluno ou aluna entrar em risco de defasagem.

# 10. Efetividade do programa: Os indicadores mostram melhora consistente ao longo do ciclo nas diferentes fases (Quartzo, Ágata, Ametista e Topázio), confirmando o impacto real do programa?

# 11. Insights e criatividade: Você pode adicionar mais insights e pontos de vista não abordados nas perguntas, utilize a criatividade e a análise dos dados para trazer sugestões para a Passos Mágicos.

# Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem?